# Document Processing

Let's see how to connect documents to our graph using Neo4j's document processing capabilities.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

## Connect Product Manual

First, let's create a function to connect a product manual to a product in our graph:

In [ ]:
def add_product_manual(product_id, file_path):
    """Add a product manual document and connect it to a product.
    
    Args:
        product_id (str): ID of the product to connect the manual to
        file_path (str): Path to the manual text file
        
    Returns:
        str: ID of the created document
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Product {id: $product_id})
        CREATE (d:Document {
            id: randomUUID(),
            type: 'manual',
            content: $content,
            created_at: datetime()
        })
        CREATE (p)-[:HAS_MANUAL]->(d)
        RETURN d.id as doc_id
        """, product_id=product_id, content=content)
        
        return result.single()['doc_id']

## Connect Support Case

Now let's create a function to connect a support case document to both a customer and a product:

In [ ]:
def add_support_case(customer_id, product_id, file_path):
    """Add a support case document and connect it to a customer and product.
    
    Args:
        customer_id (str): ID of the customer who filed the case
        product_id (str): ID of the product the case is about
        file_path (str): Path to the support case text file
        
    Returns:
        str: ID of the created document
    """
    with open(file_path, 'r') as f:
        content = f.read()
    
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {id: $customer_id})
        MATCH (p:Product {id: $product_id})
        CREATE (d:Document {
            id: randomUUID(),
            type: 'support_case',
            content: $content,
            created_at: datetime()
        })
        CREATE (c)-[:HAS_CASE]->(d)
        CREATE (d)-[:ABOUT]->(p)
        RETURN d.id as doc_id
        """, customer_id=customer_id, product_id=product_id, content=content)
        
        return result.single()['doc_id']

## Query Connected Documents

Finally, let's create functions to query documents connected to products and customers:

In [ ]:
def get_product_documents(product_id):
    """Get all documents connected to a product.
    
    Args:
        product_id (str): ID of the product to get documents for
        
    Returns:
        list: List of documents with their type and content
    """
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Product {id: $product_id})
        MATCH (p)-[:HAS_MANUAL|ABOUT]-(d:Document)
        RETURN d.type as doc_type,
               d.content as content
        """, product_id=product_id)
        
        return list(result)

def get_customer_documents(customer_id):
    """Get all documents connected to a customer.
    
    Args:
        customer_id (str): ID of the customer to get documents for
        
    Returns:
        list: List of documents with their type and content
    """
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {id: $customer_id})
        MATCH (c)-[:HAS_CASE]->(d:Document)
        RETURN d.type as doc_type,
               d.content as content
        """, customer_id=customer_id)
        
        return list(result)

## Example Usage

Here's how to use these functions:

1. Add a product manual:
```python
manual_id = add_product_manual('product-123', 'path/to/manual.txt')
```

2. Add a support case:
```python
case_id = add_support_case('customer-456', 'product-123', 'path/to/case.txt')
```

3. Query documents:
```python
product_docs = get_product_documents('product-123')
customer_docs = get_customer_documents('customer-456')
```

## Next Steps

Now that we have functions to connect documents to our graph, let's explore GraphRAG patterns!